In [12]:
import pandas as pd
import numpy as np
import requests
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from bs4 import BeautifulSoup 

In [11]:
!pip install geopy
!pip install bs4

     |████████████████████████████████| 122kB 23.7MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [34]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_url = requests.get(url).text

In [19]:
soup = BeautifulSoup(wiki_url,'html5lib')

[                                                    0  \
 0                                     M1ANot assigned   
 1                     M1BScarborough(Malvern / Rouge)   
 2   M1CScarborough(Rouge Hill / Port Union / Highl...   
 3   M1EScarborough(Guildwood / Morningside / West ...   
 4                              M1GScarborough(Woburn)   
 5                           M1HScarborough(Cedarbrae)   
 6                 M1JScarborough(Scarborough Village)   
 7   M1KScarborough(Kennedy Park / Ionview / East B...   
 8   M1LScarborough(Golden Mile / Clairlea / Oakridge)   
 9   M1MScarborough(Cliffside / Cliffcrest / Scarbo...   
 10       M1NScarborough(Birch Cliff / Cliffside West)   
 11  M1PScarborough(Dorset Park / Wexford Heights /...   
 12                 M1RScarborough(Wexford / Maryvale)   
 13                          M1SScarborough(Agincourt)   
 14  M1TScarborough(Clarks Corners / Tam O'Shanter ...   
 15  M1VScarborough(Milliken / Agincourt North / St...   
 16     M1WSca

In [21]:
len(wiki_data), type(wiki_data)

(3, list)

In [40]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)

df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [30]:
df.groupby(['PostalCode']).first()

,index,Borough,Neighborhood
PostalCode,,,
M1B,6,Scarborough,"Malvern, Rouge"
M1C,12,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,18,Scarborough,"Guildwood, Morningside, West Hill"
M1G,22,Scarborough,Woburn
M1H,26,Scarborough,Cedarbrae
...,...,...,...
M9N,64,York,Weston
M9P,70,Etobicoke,Westmount
M9R,77,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."


In [31]:
len(df['PostalCode'].unique())

103

In [32]:
df[df['Borough'] == 'Not assigned']

,index,PostalCode,Borough,Neighborhood


In [33]:
df.shape

(103, 4)

## Part 2

In [36]:
!pip install geocoder

     |████████████████████████████████| 102kB 14.0MB/s ta 0:00:01


In [37]:
import geocoder

In [38]:
data = pd.read_csv("https://cocl.us/Geospatial_data")
data.reset
data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [41]:
print("The shape of our wiki data is: ", df.shape)
print("the shape of our csv data is: ", data.shape)

The shape of our wiki data is:  (103, 3)
the shape of our csv data is:  (103, 3)


In [42]:
df.dtypes

PostalCode      object
Borough         object
Neighborhood    object
dtype: object

In [43]:
data.dtypes

Postal Code     object
Latitude       float64
Longitude      float64
dtype: object

In [46]:
combined_data = df.join(data.set_index('Postal Code'), on='PostalCode', how='inner')
combined_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [47]:
combined_data.shape

(103, 5)